# Логистическая регрессия. Практика

В этом задании вам предлагается спрогнозировать, купит клиент велосипед или нет, обучив логистическую регрессию.

In [ ]:
# подключим библиотеки
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings("ignore")

In [ ]:
# считаем данные
data = pd.read_csv('https://raw.githubusercontent.com/evgpat/edu_stepik_practical_ml/main/datasets/bike_buyers_clean.csv')

In [ ]:
# выводим первые 5 строк датафрейма
data.head(5)

,ID,Marital Status,Gender,Income,Children,Education,Occupation,Home Owner,Cars,Commute Distance,Region,Age,Purchased Bike
0,12496,Married,Female,40000,1,Bachelors,Skilled Manual,Yes,0,0-1 Miles,Europe,42,No
1,24107,Married,Male,30000,3,Partial College,Clerical,Yes,1,0-1 Miles,Europe,43,No
2,14177,Married,Male,80000,5,Partial College,Professional,No,2,2-5 Miles,Europe,60,No
3,24381,Single,Male,70000,0,Bachelors,Professional,Yes,1,5-10 Miles,Pacific,41,Yes
4,25597,Single,Male,30000,0,Bachelors,Clerical,No,0,0-1 Miles,Europe,36,Yes


In [ ]:
data['Gender'] = data['Gender'].map({'Female' : 0, 'Male' : 1})
data['Gender']

0      0
1      1
2      1
3      1
4      1
      ..
995    1
996    1
997    1
998    1
999    1
Name: Gender, Length: 1000, dtype: int64

Выведите статистики по категориальным признакам, чтобы посмотреть, сколько категорий в каждом категориальном (нечисловом) признаке.

Для этого можно воспользоваться методом `describe` из библиотеки pandas со значением параметра `include = 'object'`.

**Вопрос:** в каком категориальном признаке встречаются три различных значения?

In [ ]:
'''for col in data.columns:
        if data[col].nunique() == 3:
            print(f"Признак {col} имеет три различных значения")'''
categorical_stats = data.describe(include=['object'])
print(categorical_stats)

       Marital Status Gender  Education    Occupation Home Owner  \
count            1000   1000       1000          1000       1000   
unique              2      2          5             5          2   
top           Married   Male  Bachelors  Professional        Yes   
freq              539    509        306           276        685   

       Commute Distance         Region Purchased Bike  
count              1000           1000           1000  
unique                5              3              2  
top           0-1 Miles  North America             No  
freq                366            508            519  


Закодируйте все категориальные столбцы с двумя категориями следующим образом:  
самая часто встречающаяся категория превращается в 1, другая в 0.

In [ ]:
binar_encoding_dict = {
    'Marital Status': {'Married': 1, 'Single': 0},
    'Gender': {'Male': 1, 'Female': 0},
    'Home Owner': {'Yes': 1, 'No': 0},
    'Purchased Bike': {'No': 1, 'Yes': 0}
}

for column, mapping in binar_encoding_dict.items():
    data[column] = data[column].map(mapping)

data = data.drop(['Education', 'Occupation', 'Commute Distance', 'Region'],  axis=1)

Удалите остальные категориальные столбцы.

**Вопрос:** сколько категориальных столбцов вы удалили?

In [ ]:
data.drop(columns=['ID'], inplace=True)

Удалите столбец `ID`, так как он по сути нечисловой.

In [ ]:
# ваш код здесь

Сформируйте матрицу объект-признак `X` и вектор `y` с целевой переменной.  
Целевая переменная - это последний столбец, `Purchased Bike`.

In [ ]:
y = data['Purchased Bike']
X = data.drop(columns=['Purchased Bike'])
X


,Marital Status,Gender,Income,Children,Home Owner,Cars,Age
0,1,0,40000,1,1,0,42
1,1,1,30000,3,1,1,43
2,1,1,80000,5,0,2,60
3,0,1,70000,0,1,1,41
4,0,1,30000,0,0,0,36
...,...,...,...,...,...,...,...
995,1,1,60000,2,1,2,54
996,0,1,70000,4,1,0,35
997,1,1,60000,2,1,0,38
998,0,1,100000,3,0,3,38


Разбейте данные на тренировочную и тестовую часть (`Xtrain`, `Xtest`, `ytrain`, `ytest`), в тест отправьте 30% данных.  
Зафиксируйте `random_state = 42`.

In [ ]:
from sklearn.model_selection import train_test_split

Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=0.3, random_state=42)

**Вопрос:** сколько объектов в матрице `Xtrain`?

In [ ]:
Xtrain.shape

(700, 7)

Почти всё готово для обучения модели!

Осталось отмасштабировать матрицу `X`, так как линейные модели чувствительны к масштабу данных.

*  Обучите на тренировочной матрице (`Xtrain`) `MinMaxScaler` из библиотеки `sklearn.preprocessing`
*  Примените масштабирование и к `Xtrain`, и к `Xtest`
*  Переведите полученные после масштабирования `np.array` обратно в pandas `dataframe`.

Полученные масштабированные матрицы назовите, как и раньше, `Xtrain` и `Xtest`.

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaler.fit(Xtrain)
Xtrain_scaled = scaler.transform(Xtrain)
Xtest_scaled = scaler.transform(Xtest)
Xtrain = pd.DataFrame(Xtrain_scaled, columns=Xtrain.columns)
Xtest = pd.DataFrame(Xtest_scaled, columns=Xtest.columns)

Теперь обучите логистическую регрессию на тренировочных данных

In [ ]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
lr.fit(Xtrain, ytrain)

LogisticRegression()

Сделайте предсказания на тренировочных и на тестовых данных.

In [ ]:
predict_train = lr.predict(Xtrain)
predict_main = lr.predict(Xtest)

Оцените значение accuracy на трейне и на тесте.

In [ ]:
from sklearn.metrics import accuracy_score

accuracy_train = accuracy_score(ytrain, predict_train)
accuracy_main = accuracy_score(ytest, predict_main)
print(accuracy_train)
accuracy_main


0.6342857142857142


0.58

Качество модели получилось невысоким, зато модель не переобучена.

Попробуем добавить новых признаков в модель, используя [PolynomialFeatures](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PolynomialFeatures.html).

Создайте полиномиальные признаки degree = 2.

Как обычно:
*  `fit` делайте на тренировочных данных
*  `transform` и на тренировочных, и на тестовых данных. Затем верните результат к формату pandas `dataframe`.

Полученные матрицы назовите, как и раньше, `Xtrain` и `Xtest`.

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
poly = PolynomialFeatures(degree=2)
Xtrain_poly = poly.fit_transform(Xtrain)
Xtest_poly = poly.fit_transform(Xtest)
Xtrain = pd.DataFrame(Xtrain_poly, columns=poly.get_feature_names_out(Xtrain.columns))
Xtest = pd.DataFrame(Xtest_poly, columns=poly.get_feature_names_out(Xtest.columns))
Xtrain

,1,Marital Status,Gender,Income,Children,Home Owner,Cars,Age,Marital Status^2,Marital Status Gender,...,Children^2,Children Home Owner,Children Cars,Children Age,Home Owner^2,Home Owner Cars,Home Owner Age,Cars^2,Cars Age,Age^2
0,1.0,1.0,1.0,0.2500,0.2,1.0,0.00,0.140625,1.0,1.0,...,0.04,0.2,0.00,0.028125,1.0,0.00,0.140625,0.0000,0.000000,0.019775
1,1.0,0.0,1.0,0.5000,0.0,0.0,0.75,0.140625,0.0,0.0,...,0.00,0.0,0.00,0.000000,0.0,0.00,0.000000,0.5625,0.105469,0.019775
2,1.0,0.0,1.0,0.1875,0.0,0.0,0.00,0.171875,0.0,0.0,...,0.00,0.0,0.00,0.000000,0.0,0.00,0.000000,0.0000,0.000000,0.029541
3,1.0,0.0,1.0,0.6250,0.0,0.0,0.75,0.109375,0.0,0.0,...,0.00,0.0,0.00,0.000000,0.0,0.00,0.000000,0.5625,0.082031,0.011963
4,1.0,1.0,1.0,0.5000,1.0,1.0,0.75,0.250000,1.0,1.0,...,1.00,1.0,0.75,0.250000,1.0,0.75,0.250000,0.5625,0.187500,0.062500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
695,1.0,1.0,1.0,0.3750,0.4,1.0,0.50,0.421875,1.0,1.0,...,0.16,0.4,0.20,0.168750,1.0,0.50,0.421875,0.2500,0.210938,0.177979
696,1.0,0.0,0.0,0.0000,0.4,1.0,0.00,0.406250,0.0,0.0,...,0.16,0.4,0.00,0.162500,1.0,0.00,0.406250,0.0000,0.000000,0.165039
697,1.0,0.0,1.0,0.1250,0.0,1.0,0.25,0.109375,0.0,0.0,...,0.00,0.0,0.00,0.000000,1.0,0.25,0.109375,0.0625,0.027344,0.011963
698,1.0,0.0,0.0,0.0000,0.4,0.0,0.25,0.671875,0.0,0.0,...,0.16,0.0,0.10,0.268750,0.0,0.00,0.000000,0.0625,0.167969,0.451416


**Вопрос:** на сколько признаков стало больше при добавлении полиномиальных признаков второй степени?

Заново обучите логистическую регрессию, уже на расширенной матрице признаков, и сделайте предсказания на трейне и тесте, а затем оцените качество (*accuracy*).

In [ ]:
lrr = LogisticRegression()
lrr.fit(Xtrain, ytrain)
y__pred = lrr.predict(Xtest)
accuracy_score(ytest, y__pred)

0.6233333333333333

**Вопрос:** на сколько повысилось качество модели на тестовых данных?  
Ответ округлите до сотых.

Появились новые требования от заказчика!

Заказчик просит, чтобы:
*  доля найденных моделью потенциальных покупателей была максимальной
*  accuracy при этом была не ниже, чем 0.6 (отклонения *accuracy* на тестовых данных на $\pm 0.05$ допустимы).

Сначала посмотрите, какие значения *recall* и *accuracy* имеют предсказния модели на тесте с классами, предсказанными по умолчанию (методом `predict`).

In [ ]:
from sklearn.metrics import recall_score, accuracy_score
rec = recall_score(ytest, y__pred)
ac = accuracy_score(ytest, y__pred)
print(rec)
ac

0.6959459459459459


0.6233333333333333

Подберём порог для перевода вероятностей в классы, чтобы оптимизировать требуемые метрики!

Разобъем тренировочные данные на трейн и валидацию, чтобы по валидационной части подбирать порог.

In [ ]:
XtrainS, Xval, ytrainS, yval = train_test_split(Xtrain, ytrain, test_size=0.3, random_state=42)

XtrainS.shape, Xval.shape

((490, 36), (210, 36))

* Обучите модель на тренировочных данных.
* Предскажите вероятности положительного класса на валидационных данных

В цикле для каждого значения порога:
*  переведите вероятности в классы
*  вычислите полноту (на валидационных данных)

Выведите на экран:

1) значение порога, дающее максимальный *recall*, при условии, что *accuracy* $\geq$ 0.6.

2) значение *recall* при этом пороге

3) значение *accuracy* для этого порога


Ищите порог на отрезке от 0 до 1 с шагом 0.01.

In [ ]:
from sklearn.metrics import recall_score

RecMax = -1
BestThr = -1
BestAcc = -1
scc = MinMaxScaler()
Xtrain_scaled = scc.fit_transform(XtrainS)
Xval_scaled = scc.transform(Xval)
poly = PolynomialFeatures(degree=2)
Xtrain_poly = poly.fit_transform(Xtrain_scaled)
Xval_poly = poly.transform(Xval_scaled)
model = LogisticRegression(penalty='l2', solver='liblinear')
model.fit(Xtrain_poly, ytrainS)
yval_probs = model.predict_proba(Xval_poly)[:, 1]
for thr in np.arange(0, 1.01, 0.01):
    # Преобразование вероятностей в классы
    yval_pred = (yval_probs >= thr).astype(int)

    # Вычисление метрик
    recall = recall_score(yval, yval_pred)
    accuracy = accuracy_score(yval, yval_pred)

    # Проверка условий и обновление лучших значений
    if accuracy >= 0.6 and recall > RecMax:
        RecMax = recall
        BestThr = thr
        BestAcc = accuracy

print(BestThr, RecMax, BestAcc)

0.38 0.8703703703703703 0.6


Теперь заново обучите модель на исходных тренировочных данных (`Xtrain`, `ytrain`), предскажите вероятности на тесте и переведите их в классы по найденному порогу.

In [ ]:
# ваш код здесь

**Вопрос:** какое значение *recall* получилось на тестовых данных после подбора порога?  
Ответ округлите до десятых.

При помощи подбора порога удалось сильно увеличить значение *recall*!  
Однако, как видно, на тесте не удалось сохранить условие $accuracy \geq 0.6$ (но в допустимые рамки уложились!)

Это свидетельство небольшого переобучения модели. Однако в сухом остатке имеет значительное увеличение полноты, что является приоритетом для заказчика.